In [1]:
! pip install jax flax einops diffusers transformers datasets wandb tqdm numpy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import jax, math, flax, cv2
import os, wandb, time, optax
from jax import Array, numpy as jnp, random as jrand
import numpy as np
from flax import nnx
from einops import rearrange
from tqdm.auto import tqdm
from typing import List, Tuple
from functools import partial
# from diffusers import AutoencoderKL
from datasets import load_dataset
from torch.utils.data import DataLoader, IterableDataset
from collections import namedtuple
import flax.traverse_util
from flax.serialization import to_state_dict
import safetensors.flax as safejax
from PIL import Image as pillow
from diffusers.models.autoencoders.autoencoder_kl import AutoencoderKL
# from transformers import AutoTokenizer, T5EncoderModel
from urllib.request import urlopen

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
rkey = jrand.key(3)
rngs = nnx.Rngs(3)
randkey = jrand.key(3)

num_devices = jax.device_count()
devices = jax.devices()
print(f"found {num_devices} JAX devices => {devices}")
for device in devices:
    print(f"{device} \n")

found 1 JAX devices => [CudaDevice(id=0)]
cuda:0 



In [35]:
class config:
    vaescale_factor = 0.13025
    batch_size = 128
    img_size = 32
    patch_size = (4, 4)
    lr = 1e-4
    mask_ratio = 0.75
    epochs = 5
    data_split = 10_000
    cfg_scale = 2.0
    vae_channels = 4
    celebv_id = "SwayStar123/CelebV-HQ"
    finevid_id = "HuggingFaceFV/finevideo"
    pd12_id = "Spawning/PD12M"
    vae_id = "madebyollin/sdxl-vae-fp16-fix"
    t5_id = "google-t5/t5-small"
    mini_data_id = "uoft-cs/cifar10"
    device_0 = jax.default_device()

In [88]:
# jax/numpy implementation of topk selection
def jnp_topk(array: Array, k: int):
    topk_tuple = namedtuple("topk", ["values", "ids"])
    array = jnp.asarray(array)
    flat = array.ravel()

    sort_indices = jnp.argpartition(flat, -k)[-k:]
    argsort = jnp.argsort(-flat[sort_indices])  # get sorting ids

    sort_idx = sort_indices[argsort]
    values = flat[sort_idx]

    idx = jnp.unravel_index(sort_idx, array.shape)

    if len(idx) == 1:
        (idx,) = idx

    return topk_tuple(values=values, ids=idx)


# save model params in safetensors file
def save_model(model: nnx.Module, file: str, dtype):
    _, state = nnx.split(model)
    params = state.filter(nnx.Param)
    state_dict = to_state_dict(params)

    state_dict = flax.traverse_util.flatten_dict(state_dict, sep=".")

    for key in list(state_dict.keys()):
        if not isinstance(state_dict[key], Array):
            state_dict[key] = jnp.array(state_dict[key])  # type: ignore

        state_dict[key] = state_dict[key].astype(dtype)  # type: ignore

    safejax.save_file(state_dict, file)  # type: ignore


def apply_mask(x: Array, mask, patch_size):
    # basically turns the masked values to 0s
    bs, c, h, w = x.shape
    numpatch_h = h // patch_size[0]
    numpatch_w = w // patch_size[1]

    mask = jnp.reshape(mask, shape=(bs, numpatch_h, numpatch_w))

    mask = jnp.expand_dims(mask, axis=1)
    mask = jnp.tile(mask, reps=(1, 1, patch_size[0], patch_size[1]))
    mask = jnp.reshape(mask, shape=(bs, 1, h, w))

    x_masked = x * mask

    return x_masked


def random_mask(bs, height, width, patch_size, mask_ratio):
    num_patches = (height // patch_size[0]) * (width // patch_size[1])
    num_patches_to_mask = int(num_patches * mask_ratio)

    rand_array = jrand.normal(randkey, shape=(bs, num_patches))
    indices = jnp.argsort(rand_array, axis=1)

    mask = jnp.ones(shape=(bs, num_patches))

    batch_mask_array = jnp.expand_dims(jnp.arange(bs), axis=1)
    # mask[batch_mask_array, indices[:, :num_patches_to_mask]] = 0
    new_mask = mask.at[batch_mask_array, indices[:, :num_patches_to_mask]].set(0)
    mask = new_mask
    mask = jnp.reshape(mask, shape=(bs, num_patches))

    return mask


def remove_masked_patches(patches: Array, mask: Array):
    mask = jnp.logical_not(jnp.bool(mask))

    bs, num_patches, embed_dim = patches.shape
    mask = jnp.expand_dims(mask, axis=-1)
    mask = jnp.broadcast_to(mask, shape=(-1, -1, embed_dim))
    mask_ids = jnp.nonzero(jnp.reshape(mask, (-1)))
    unmasked_patches = jnp.reshape(patches, shape=-1)
    unmasked_patches = jnp.take(unmasked_patches, mask_ids[0]).reshape(
        bs, -1, embed_dim
    )

    return unmasked_patches


def add_masked_patches(patches: Array, mask: Array):
    # Ensure mask is a boolean tensor
    mask = jnp.bool(mask)

    bs, num_patches, embed_dim = mask.shape[0], mask.shape[1], patches.shape[-1]

    # Create a tensor of zeros with the same shape and dtype as the patches tensor
    full_patches = jnp.zeros(shape=(bs, num_patches, embed_dim))

    # Iterate over each batch and place unmasked patches back in their original positions
    for b in range(bs):
        # Use the mask to place unmasked patches back in the correct positions
        full_patches[b, mask[b]] = patches[b].astype(full_patches.dtype)

    return full_patches


## nov_14_0453

# T5 text encoder
# t5_tokenizer = AutoTokenizer.from_pretrained(config.t5_id)
# t5_model = T5EncoderModel.from_pretrained(config.t5_id)


# def text_t5_encode(text_input: str, tokenizer=t5_tokenizer, model=t5_model):
#     input_ids = tokenizer(text_input, return_tensors="np").input_ids  # Batch size 1
#     outputs = model(input_ids=input_ids)
#     last_hidden_states = outputs.last_hidden_state

#     return last_hidden_states

In [36]:
## data loading
image_data = load_dataset(
    config.mini_data_id, streaming=True, split="train", trust_remote_code=True
).take(config.data_split)

vae = AutoencoderKL.from_pretrained(config.vae_id)#.to(config.device_0)

def load_image(image):
    # image = pillow.open(urlopen(url=url))
    img_array = np.array(image)
    resized = cv2.resize(img_array, dsize=(config.img_size, config.img_size))

    return resized / 255.0

In [37]:
import torch
randkey = jrand.key(3)


# class ImageData(IterableDataset):
#     def __init__(self, dataset=image_data):
#         super().__init__()
#         self.dataset = dataset

#     def __len__(self):
#         return config.data_split

#     def __iter__(self):
#         for sample in self.dataset:
#             image = sample["img_url"]  # type: ignore
#             image = load_image(image)
#             img_latents = vae.encode(image)
#             img_latents = img_latents.numpy()
#             caption_encoded = text_t5_encode(sample["caption"])  # type: ignore

#             image = jnp.array(image)
#             text_encoding = jnp.array(caption_encoded)

#             yield image, text_encoding


class ImageClassData(IterableDataset):
    def __init__(self, dataset=image_data):
        super().__init__()
        self.dataset = dataset

    def __len__(self):
        return config.data_split

    def __iter__(self):
        for sample in self.dataset:
            image = sample["img"]  # type: ignore
            image = load_image(image)
            # img_latents = vae.encode(torch.tensor(image))
            # img_latents = img_latents.

            image = jnp.array(image)
            label = jnp.array(sample["label"])

            yield image, label

def jax_collate(batch):
    images, labels = zip(*batch)
    batch = (jnp.array(images), jnp.array(labels))
    batch = jax.tree_util.tree_map(jnp.array, batch)

    return batch

dataset = ImageClassData()

train_loader = DataLoader(dataset, batch_size=4, collate_fn=jax_collate)

iv = next(iter(train_loader))
iv[0].shape

(4, 32, 32, 3)

In [38]:
iv[1]

Array([0, 6, 0, 2], dtype=int32)

In [39]:
# modulation with shift and scale
def modulate(x_array: Array, shift, scale) -> Array:
    x = x_array * scale.unsqueeze(1)
    x = x + shift.unsqueeze(1)

    return x


# equivalnet of F.lineat
def linear(array: Array, weight: Array, bias: Array | None = None) -> Array:
    out = jnp.dot(array, weight)

    if bias is not None:
        out += bias

    return out

In [25]:
# Adapted from https://github.com/facebookresearch/mae/blob/main/util/pos_embed.py
def get_2d_sincos_pos_embed(embed_dim, grid_size, cls_token=False, extra_tokens=0):
    """
    grid_size: int of the grid height and width
    return:
    pos_embed: [grid_size*grid_size, embed_dim] or [1+grid_size*grid_size, embed_dim] (w/ or w/o cls_token)
    """
    grid_h = jnp.arange(grid_size, dtype=jnp.float32)
    grid_w = jnp.arange(grid_size, dtype=jnp.float32)
    grid = jnp.meshgrid(grid_w, grid_h)  # here w goes first
    grid = jnp.stack(grid, axis=0)

    grid = grid.reshape([2, 1, grid_size, grid_size])
    pos_embed = get_2d_sincos_pos_embed_from_grid(embed_dim, grid)
    if cls_token and extra_tokens > 0:
        pos_embed = jnp.concatenate(
            [jnp.zeros([extra_tokens, embed_dim]), pos_embed], axis=0
        )
    return pos_embed


def get_2d_sincos_pos_embed_from_grid(embed_dim, grid):
    assert embed_dim % 2 == 0

    # use half of dimensions to encode grid_h
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0])  # (H*W, D/2)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1])  # (H*W, D/2)

    emb = jnp.concatenate([emb_h, emb_w], axis=1)  # (H*W, D)
    return emb


def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """
    assert embed_dim % 2 == 0
    omega = jnp.arange(embed_dim // 2, dtype=jnp.float64)
    omega /= embed_dim / 2.0
    omega = 1.0 / 10000**omega  # (D/2,)

    pos = pos.reshape(-1)  # (M,)
    out = jnp.einsum("m,d->md", pos, omega)  # (M, D/2), outer product

    emb_sin = jnp.sin(out)  # (M, D/2)
    emb_cos = jnp.cos(out)  # (M, D/2)

    emb = jnp.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)

    return emb

In [41]:
# input patchify layer, 2D image to patches
class PatchEmbed(nnx.Module):
    def __init__(
        self,
        rngs: nnx.Rngs = rngs,
        patch_size=(4,4),
        img_size=(32, 32),
        in_chan: int = 3,
        embed_dim: int = 768,
    ):
        super().__init__()
        self.patch_size = (patch_size, patch_size)
        self.img_size = img_size
        self.gridsize = tuple([s // p for s, p in zip(img_size, patch_size)])
        self.num_patches = self.gridsize[0] * self.gridsize[1]
        linear_init = nnx.initializers.constant(0)

        self.conv_project = nnx.Conv(
            in_chan,
            embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            rngs=rngs,
        )

    def __call__(self, img: Array) -> Array:
        x = self.conv_project(img)
        x = jnp.reshape(x, shape=(x.shape[0], -1))
        print(f'patchembed shape => {x.shape}')
        return x

patchem = PatchEmbed()
patchimg = patchem(iv[0])

# embeds a flat vector
class VectorEmbedder(nnx.Module):
    def __init__(self, input_dim, hidden_size, rngs=rngs):
        super().__init__()
        self.linear_1 = nnx.Linear(input_dim, hidden_size, rngs=rngs)
        self.linear_2 = nnx.Linear(hidden_size, hidden_size, rngs=rngs)

    def __call__(self, x: Array):
        x = nnx.silu(self.linear_1(x))
        x = self.linear_2(x)

        return x


class TimestepEmbedder(nnx.Module):
    def __init__(self, hidden_size, freq_embed_size=256):
        super().__init__()
        self.lin_1 = nnx.Linear(freq_embed_size, hidden_size, rngs=rngs)
        self.lin_2 = nnx.Linear(hidden_size, hidden_size, rngs=rngs)
        self.freq_embed_size = freq_embed_size

    @staticmethod
    def timestep_embedding(time_array: Array, dim, max_period=10000):
        half = dim // 2
        freqs = jnp.exp(-math.log(max_period) * jnp.arange(0, half) / half).to_device(
            time_array.device
        )
        args = jnp.float_(time_array[:, None]) * freqs[None]

        embedding = jnp.concat([jnp.cos(args), jnp.sin(args)], axis=-1)
        if dim % 2:
            embedding = jnp.concat(
                [embedding, jnp.zeros_like(embedding[:, :1])], axis=-1
            )

        return embedding

    def __call__(self, x: Array) -> Array:
        t_freq = self.timestep_embedding(x, self.freq_embed_size)
        t_embed = nnx.silu(self.lin_1(t_freq))

        return self.lin_2(t_embed)


class LabelEmbedder(nnx.Module):
    def __init__(self, num_classes, hidden_size, drop):
        super().__init__()
        use_cfg_embeddings = drop > 0
        self.embedding_table = nnx.Embed(
            num_classes + use_cfg_embeddings, hidden_size, rngs=rngs
        )
        self.num_classes = num_classes
        self.dropout = drop

    def token_drop(self, labels, force_drop_ids=None) -> Array:
        if force_drop_ids is None:
            drop_ids = jrand.normal(key=rkey, shape=labels.shape[0]).to_device(
                labels.device
            )
        else:
            drop_ids = force_drop_ids == 1

        labels = jnp.where(drop_ids, self.num_classes, labels)

        return labels

    def __call__(self, labels, train: bool, force_drop_ids=None) -> Array:
        use_drop = self.dropout > 0
        if (train and use_drop) or (force_drop_ids is not None):
            labels = self.token_drop(labels, force_drop_ids)

        label_embeds = self.embedding_table(labels)

        return label_embeds


class CaptionEmbedder(nnx.Module):
    def __init__(self, cap_embed_dim, embed_dim):
        super().__init__()
        self.linear_1 = nnx.Linear(cap_embed_dim, embed_dim, rngs=rngs)
        self.linear_2 = nnx.Linear(embed_dim, embed_dim, rngs=rngs)

    def __call__(self, x: Array) -> Array:
        x = nnx.silu(self.linear_1(x))
        x = self.linear_2(x)

        return x

# self attention block
class SelfAttention(nnx.Module):
    def __init__(self, attn_heads, embed_dim, rngs: nnx.Rngs, drop=0.1):
        super().__init__()
        self.attn_heads = attn_heads
        self.head_dim = embed_dim // attn_heads

        linear_init = nnx.initializers.xavier_uniform()
        linear_bias_init = nnx.initializers.constant(0)

        self.q_linear = nnx.Linear(
            embed_dim,
            embed_dim,
            rngs=rngs,
            bias_init=linear_bias_init,
            kernel_init=linear_init,
        )
        self.k_linear = nnx.Linear(embed_dim, embed_dim, rngs=rngs)
        self.v_linear = nnx.Linear(embed_dim, embed_dim, rngs=rngs)

        self.outproject = nnx.Linear(
            embed_dim, embed_dim, rngs=rngs, bias_init=linear_bias_init
        )
        self.dropout = nnx.Dropout(drop, rngs=rngs)

    def __call__(self, x_input: jax.Array) -> jax.Array:
        q = self.q_linear(x_input)
        k = self.k_linear(x_input)
        v = self.v_linear(x_input)

        q, k, v = map(
            lambda x: rearrange(x, "b l (h d) -> b h l d", h=self.attn_heads), (q, k, v)
        )

        qk = q @ jnp.matrix_transpose(k)
        attn_logits = qk / math.sqrt(self.head_dim)  # attention computation

        attn_score = nnx.softmax(attn_logits, axis=-1)
        attn_output = attn_score @ v

        output = rearrange(attn_output, "b h l d -> b l (h d)")
        output = self.dropout(self.outproject(output))
        print(f"attn out shape => {output.shape}")
        return output


class CrossAttention(nnx.Module):
    def __init__(self, attn_heads, embed_dim, cond_dim, rngs: nnx.Rngs, drop=0.1):
        super().__init__()
        self.attn_heads = attn_heads
        self.head_dim = embed_dim // attn_heads

        linear_init = nnx.initializers.xavier_uniform()
        linear_bias_init = nnx.initializers.constant(0)

        self.q_linear = nnx.Linear(
            embed_dim,
            embed_dim,
            rngs=rngs,
            bias_init=linear_bias_init,
            kernel_init=linear_init,
        )

        self.k_linear = nnx.Linear(cond_dim, embed_dim, rngs=rngs)
        self.v_linear = nnx.Linear(cond_dim, embed_dim, rngs=rngs)

        self.outproject = nnx.Linear(
            embed_dim, embed_dim, rngs=rngs, bias_init=linear_bias_init
        )
        self.dropout = nnx.Dropout(drop, rngs=rngs)

    def __call__(self, x_input: jax.Array, y_cond: Array) -> jax.Array:
        q = self.q_linear(x_input)
        k = self.k_linear(y_cond)
        v = self.v_linear(y_cond)

        q, k, v = map(
            lambda x: rearrange(x, "b l (h d) -> b h l d", h=self.attn_heads), (q, k, v)
        )

        qk = q @ jnp.matrix_transpose(k)
        attn_logits = qk / math.sqrt(self.head_dim)  # attention computation

        attn_score = nnx.softmax(attn_logits, axis=-1)
        attn_output = attn_score @ v

        output = rearrange(attn_output, "b h l d -> b l (h d)")
        output = self.dropout(self.outproject(output))
        print(f"attn out shape => {output.shape}")
        return output


########################
# Patch Mixer components
########################


class EncoderMLP(nnx.Module):
    def __init__(self, hidden_size, rngs: nnx.Rngs, dropout=0.1):
        super().__init__()
        self.layernorm = nnx.LayerNorm(hidden_size, rngs=rngs)

        self.linear1 = nnx.Linear(hidden_size, 2 * hidden_size, rngs=rngs)
        self.linear2 = nnx.Linear(2 * hidden_size, hidden_size, rngs=rngs)
        self.dropout = nnx.Dropout(dropout, rngs=rngs)

    def __call__(self, x_input: jax.Array) -> jax.Array:
        x = self.layernorm(x_input)
        x = nnx.silu(self.linear1(x))
        x = self.linear2(x)

        return x


class TransformerEncoderBlock(nnx.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.layernorm = nnx.LayerNorm(embed_dim, epsilon=1e-6, rngs=rngs)
        self.self_attention = SelfAttention(num_heads, embed_dim, rngs=rngs)
        self.mlp_layer = EncoderMLP(embed_dim, rngs=rngs)

    def __call__(self, x: Array):
        x = x + self.layernorm(self.self_attention(x))
        x = x + self.layernorm(self.mlp_layer(x))

        return x

patchembed shape => (4, 49152)


In [50]:


class SimpleMLP(nnx.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.linear_1 = nnx.Linear(embed_dim, embed_dim, rngs=rngs)
        self.linear_2 = nnx.Linear(embed_dim, embed_dim, rngs=rngs)

    def __call__(self, x: Array) -> Array:
        x = nnx.silu(self.linear_1(x))
        x = self.linear_2(x)

        return x


# Pool + MLP for (MHA + MLP)
class PoolMLP(nnx.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.linear_1 = nnx.Linear(embed_dim, embed_dim, rngs=rngs)
        self.linear_2 = nnx.Linear(embed_dim, embed_dim, rngs=rngs)

    def __call__(self, x: Array) -> Array:
        x = nnx.avg_pool(x, 1, 1)
        x = jnp.reshape(x, shape=(x.shape[0], -1))
        x = nnx.gelu(self.linear_1(x))
        x = self.linear_2(x)

        return x


#### MoE Gate
class MoEGate(nnx.Module):
    def __init__(
        self, embed_dim, num_experts=8, experts_per_token=2, aux_loss_alpha=0.01
    ):
        super().__init__()
        self.top_k = experts_per_token
        self.routed_exoerts = num_experts
        self.score_func = "softmax"
        self.alpha = aux_loss_alpha
        self.seq_aux = False

        # top_k selection algo
        self.norm_topk_prob = False
        self.gating_dim = embed_dim
        param_init = nnx.initializers.he_uniform()
        self.weight = nnx.Param(jnp.empty((self.routed_exoerts, self.gating_dim)))
        # self.linear_gate = nnx.Linear()

    def __call__(self, hidden_states: Array) -> Tuple:
        bsize, seq_len, h = hidden_states.shape
        # gating score
        hidden_states = jnp.reshape(hidden_states, (-1, h))
        logits = jnp.dot(hidden_states, self.weight["Array"])
        scores = nnx.softmax(logits, axis=-1)

        topk_idx, topk_weight = jax.lax.top_k(scores, k=self.top_k)

        # normalize to sum to 1
        if self.top_k > 1 and self.norm_topk_prob:
            denominator = jnp.sum(topk_weight, axis=-1, keepdims=True) + 1e-20
            topk_weight = topk_weight / denominator

        # expert level computation of auxiliary loss
        # always compute topk based on naive greedy topk method
        if self.train and self.alpha > 0.0:
            scores_for_aux = scores
            aux_topk = self.top_k
            aux_loss = 0.0

            topk_idx_for_auxloss = jnp.reshape(topk_idx, (bsize, -1))
            if self.seq_aux:
                scores_for_seq_aux = jnp.reshape(scores_for_aux, (bsize, seq_len, -1))
                ce = jnp.zeros(
                    (bsize, self.routed_exoerts), device=hidden_states.device
                )
                ones_add = jnp.ones((bsize, seq_len * aux_topk))
                ce = jnp.add.at(ce, 1, ones_add)
                ce /= seq_len * aux_topk / self.routed_exoerts

                aux_loss = (ce * scores_for_seq_aux.mean(axis=1)).sum(
                    axis=1
                ).mean() * self.alpha

            else:
                mask_ce = nnx.one_hot(
                    jnp.reshape(topk_idx_for_auxloss, (-1)),
                    num_classes=self.routed_exoerts,
                )
                ce = mask_ce.astype(jnp.float32).mean(0)
                pi = scores_for_aux.mean()
                fi = ce * self.routed_exoerts
                aux_loss = (pi * fi).sum() * self.alpha

        else:
            aux_loss = None

        print(f"gate shape => {topk_weight.shape}")

        return topk_idx, topk_weight, aux_loss

In [52]:
# mixture of experts MLP layer
class MoEMLP(nnx.Module):
    def __init__(self, hidden_size, intersize, pretrain_tp=2):
        self.hidden_size = hidden_size
        self.intersize = intersize
        self.pretrain_tp = pretrain_tp
        self.gate_project = nnx.Linear(
            self.hidden_size, self.intersize, use_bias=False, rngs=rngs
        )
        self.up_project = nnx.Linear(
            self.hidden_size, self.intersize, use_bias=False, rngs=rngs
        )
        self.down_project = nnx.Linear(
            self.intersize, self.hidden_size, use_bias=False, rngs=rngs
        )

    def __call__(self, x_input: Array):
        down_proj = None

        if self.pretrain_tp > 1:
            w_slice = self.intersize // self.pretrain_tp
            gate_slices = jnp.split(self.gate_project.kernel[1:], w_slice, axis=0)
            up_slices = jnp.split(self.up_project.kernel[1:], w_slice, axis=0)
            down_slices = jnp.split(self.down_project.kernel[1:], w_slice, axis=1)

            gate_proj = jnp.concat(
                [linear(x_input, gate_slices[k]) for k in range(self.pretrain_tp)],
                axis=-1,
            )
            up_proj = jnp.concat(
                [linear(x_input, up_slices[k]) for k in range(self.pretrain_tp)],
                axis=-1,
            )
            inter_states = jnp.split((nnx.silu(gate_proj) * up_proj), w_slice, axis=-1)
            down_proj = [
                linear(inter_states[k], down_slices[k]) for k in range(self.pretrain_tp)
            ]
            down_proj = sum(down_proj)

        else:
            activated_x = nnx.silu(self.gate_project(x_input)) * self.up_project(
                x_input
            )
            down_proj = self.down_project(activated_x)


        print(f"moe mlp shape => {down_proj.shape}")

        return down_proj


class SparseMoEBlock(nnx.Module):
    def __init__(
        self,
        embed_dim,
        mlp_ratio=4,
        num_experts=8,
        experts_per_token=2,
        train: bool = True,
        rngs=rngs,
    ):
        super().__init__()
        self.experts_pertoken = experts_per_token
        self.expert_models = [
            MoEMLP(hidden_size=embed_dim, intersize=mlp_ratio * embed_dim)
            for _ in range(num_experts)
        ]
        # self.experts = nnx.Sequential(*self.expert_models)
        self.router_gate = MoEGate(embed_dim, num_experts)
        self.n_shared_experts = 2
        self.training = train

        if self.n_shared_experts is not None:
            intermediate_size = embed_dim * self.n_shared_experts
            self.shared_experts = MoEMLP(
                hidden_size=embed_dim, intersize=intermediate_size
            )

    def __call__(self, hidden_states: Array):
        identity = hidden_states
        og_shape = hidden_states.shape
        topk_idx, topk_weight, aux_loss = self.router_gate(hidden_states)
        y = jrand.normal(rkey, shape=og_shape)  # init as random array

        hidden_states = jnp.reshape(hidden_states, (-1, hidden_states.shape[1:]))
        flat_topk_idx = jnp.reshape(topk_idx, shape=(-1))

        if self.train:
            hidden_states = jnp.repeat(
                hidden_states, repeats=self.experts_pertoken, axis=0
            )
            y = jnp.empty_like(hidden_states, dtype=hidden_states.dtype)

            for k, expert in enumerate(self.expert_models):
                y[flat_topk_idx == k] = expert(hidden_states[flat_topk_idx == k]).astype(hidden_states.dtype)  # type: ignore

            y = jnp.reshape(y, shape=(*topk_weight.shape, -1)) * jnp.expand_dims(
                topk_weight, axis=-1
            ).sum(axis=1)
            y = jnp.reshape(y, shape=(og_shape))

            # TODO: Auxiliary loss add

        else:
            y = None

        if self.shared_experts is not None:
            y = y + self.shared_experts(identity)  # type: ignore

        # if aux_loss is not None:
        #     return y, aux_loss
        print(f'sparse moe shape =>{y.shape}')
        return y

    def moe_infer(self, x_input):
        pass

moeblock = SparseMoEBlock(embed_dim=1024)

# moeblock(iv[0])

In [102]:

###############
# DiT blocks_ #
###############


class DiTBlock(nnx.Module):
    def __init__(self, hidden_size=1024, num_heads=6):
        super().__init__()

        # initializations
        linear_init = nnx.initializers.xavier_uniform()
        lnbias_init = nnx.initializers.constant(0)
        lnweight_init = nnx.initializers.constant(0)

        self.norm_1 = nnx.LayerNorm(
            hidden_size, epsilon=1e-6, rngs=rngs, bias_init=lnbias_init
        )
        self.attention = SelfAttention(num_heads, hidden_size, rngs=rngs)
        self.norm_2 = nnx.LayerNorm(hidden_size, epsilon=1e-6, rngs=rngs)

        self.adaln_linear = nnx.Linear(
            in_features=hidden_size,
            out_features=hidden_size,
            use_bias=True,
            # bias_init=linear_init,
            rngs=rngs,
            # kernel_init=lnweight_init,
        )
        self.moe_block = SparseMoEBlock(hidden_size)
        print('dit block online')

    def __call__(self, x_img: Array):
        x_input = self.adaln_linear(nnx.silu(x_img))
        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = (
            jnp.array_split(x_input, 6)
        )

        attn_mod_x = self.attention(
            modulate(self.norm_1(x_input), shift_msa, scale_msa)
        )
        x = x_input + jnp.expand_dims(gate_msa, 1) * attn_mod_x

        mlp_mod_x = self.moe_block(modulate(self.norm_2(x), shift_mlp, scale_mlp))
        x = x + jnp.expand_dims(gate_mlp, 1) * mlp_mod_x
        print(f'x dit block {type(x)} {x.shape}')
        return x


class FinalMLP(nnx.Module):
    def __init__(self, hidden_size, patch_size, out_channels):
        super().__init__()
        # linear_init = nnx.initializers.xavier_uniform()
        linear_init = nnx.initializers.constant(0)

        self.norm_final = nnx.LayerNorm(hidden_size, epsilon=1e-6, rngs=rngs)
        self.linear = nnx.Linear(
            hidden_size,
            patch_size * patch_size * out_channels,
            rngs=rngs,
            kernel_init=linear_init,
            bias_init=linear_init,
        )
        self.adaln_linear = nnx.Linear(
            hidden_size,
            2 * hidden_size,
            rngs=rngs,
            kernel_init=linear_init,
            bias_init=linear_init,
        )

    def __call__(self, x_input: Array, cond: Array):
        linear_cond = nnx.silu(self.adaln_linear(cond))
        shift, scale = jnp.array_split(linear_cond, 2, axis=1)

        x = modulate(self.norm_final(x_input), shift, scale)
        x = self.linear(x)
        print(f'final dit mlp {type(x)} {x.shape}')
        return


class DiTBackbone(nnx.Module):
    def __init__(
        self,
        patch_size=(4,4),
        in_channels=3,
        hidden_size=1024,
        depth=4,
        attn_heads=6,
        learn_sigma=False,
    ):
        super().__init__()
        self.learn_sigma = learn_sigma
        self.in_chan = in_channels
        self.out_channels = in_channels * 2 if learn_sigma else in_channels
        self.patch_size = patch_size
        self.attn_heads = attn_heads

        self.img_embedder = PatchEmbed(
            img_size=(config.img_size, config.img_size), in_chan=in_channels, embed_dim=hidden_size
        )
        self.time_embedder = TimestepEmbedder(hidden_size)

        num_patches = self.img_embedder.num_patches

        self.pos_embed = nnx.Param(jnp.zeros(shape=(1, num_patches, hidden_size)))
        pos_embed = get_2d_sincos_pos_embed(
            self.pos_embed.value.shape[-1], int(self.img_embedder.num_patches**0.5)
        )
        sincos2d_data = jnp.expand_dims(pos_embed.astype(jnp.float32), axis=0)
        print(f'sincos {type(sincos2d_data)} {sincos2d_data.shape}')
        # self.pos_embed.value.copy_from(sincos2d_data)  # type: ignore
        self.pos_embed.value = jnp.copy(sincos2d_data)

        dit_blocks = [DiTBlock(hidden_size, num_heads=attn_heads) for _ in tqdm(range(depth))]
        self.final_mlp = FinalMLP(hidden_size, patch_size[0], self.out_channels)
        self.dit_layers = nnx.Sequential(*dit_blocks)
        print('ditbackbone online')

    def unpatchify(self, x: Array) -> Array:
        c = self.out_channels
        p = self.img_embedder.patch_size[0]
        h = w = int(x.shape[1] ** 0.5)
        assert h * w == x.shape[1]

        x = jnp.reshape(x, shape=(x.shape[0], h, w, p, p, c))
        x = jnp.einsum("nhwpqc->nchpwq", x)
        img = jnp.reshape(x, shape=(x.shape[0], c, h * p, w * p))

        return img

    def __call__(self, x: Array, t: Array, y_cond: Array):
        x = self.img_embedder(x) + self.pos_embed
        t_embed = self.time_embedder(t)

        cond = t_embed + y_cond
        x = self.dit_layers(x, cond)
        x = self.final_mlp(x, cond)  # type: ignore
        x = self.unpatchify(x)

        print(f'ditback out -> {x.shape}')

        return x

    def cfg_forward(self, x_img, t, y_cond, cfg_scale):
        half = x_img[: len(x_img) // 2]
        combined = jnp.concat([half, half], axis=0)
        model_out = self.__call__(combined, t, y_cond)

        eps, rest = model_out[:, :3], model_out[:, 3:]
        cond_eps, uncond_eps = jnp.split(eps, len(eps) // 2, axis=0)

        half_eps = uncond_eps + cfg_scale * (cond_eps - uncond_eps)
        eps = jnp.concat([half_eps, half_eps], axis=0)
        cfg_out = jnp.concat([eps, rest], axis=1)

        return cfg_out

backbone = DiTBackbone()


sincos <class 'jaxlib.xla_extension.ArrayImpl'> (1, 64, 1024)


/usr/local/lib/python3.10/dist-packages/jax/_src/numpy/lax_numpy.py:5037: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in arange is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  output = _arange(start, stop=stop, step=step, dtype=dtype)


  0%|          | 0/4 [00:00<?, ?it/s]

dit block online
dit block online
dit block online
dit block online
ditbackbone online


In [70]:
# backbone(iv[0]).shape

In [71]:
# patch mixer module
class PatchMixer(nnx.Module):
    def __init__(self, embed_dim, attn_heads, n_layers=2):
        super().__init__()
        layers = [
            TransformerEncoderBlock(embed_dim, attn_heads) for _ in range(n_layers)
        ]
        self.encoder_layers = nnx.Sequential(*layers)

    def __call__(self, x: Array) -> Array:
        x = self.encoder_layers(x)
        return x

In [106]:
#####################
# Full Microdit model
####################
class MicroDiT(nnx.Module):
    def __init__(
        self,
        inchannels,
        patch_size,
        embed_dim,
        num_layers,
        attn_heads,
        mlp_dim,
        cond_embed_dim,
        num_experts=4,
        active_experts=2,
        dropout=0.1,
        patchmix_layers=2,
        rngs=rngs,
        num_classes=10,
    ):
        super().__init__()
        self.patch_size = patch_size
        self.embed_dim = embed_dim

        self.patch_embedder = PatchEmbed(
            rngs=rngs, patch_size=patch_size, in_chan=inchannels, embed_dim=embed_dim
        )

        # conditioning layers
        self.time_embedder = TimestepEmbedder(embed_dim)
        self.cap_embedder = CaptionEmbedder(cond_embed_dim, embed_dim)
        self.label_embedder = LabelEmbedder(
            num_classes=num_classes, hidden_size=embed_dim, drop=dropout)
        self.cond_attention = CrossAttention(
            attn_heads, embed_dim, cond_embed_dim, rngs=rngs
        )
        self.cond_mlp = SimpleMLP(embed_dim)

        # pooling layer
        self.pool_mlp = PoolMLP(embed_dim)

        self.linear = nnx.Linear(self.embed_dim, self.embed_dim, rngs=rngs)

        self.patch_mixer = PatchMixer(embed_dim, attn_heads, patchmix_layers)
        self.ditbackbone = DiTBackbone(
            patch_size=patch_size,
            in_channels=inchannels,
            hidden_size=embed_dim,
            depth=num_layers,
        )

        self.outlin_1 = nnx.Linear(embed_dim, embed_dim, rngs=rngs)
        self.final_linear = nnx.Linear(
            embed_dim, patch_size[0] * patch_size[1] * inchannels, rngs=rngs
        )

    def __call__(self, x: Array, t: Array, y_cap: Array, mask=None):
        bsize, channels, height, width = x.shape
        psize_h, psize_w = self.patch_size

        x = self.patch_embedder(x)

        pos_embed = get_2d_sincos_pos_embed(self.embed_dim, height // psize_h)
        pos_embed = jnp.expand_dims(pos_embed, axis=0)
        pos_embed = jnp.broadcast_to(pos_embed, (bsize, -1, -1))
        x = x + pos_embed

        # cond_embed = self.cap_embedder(y_cap) # (b, embdim)
        cond_embed = self.label_embedder(y_cap)
        time_embed = self.time_embedder(t)
        time_embed_unsqueeze = jnp.expand_dims(time_embed, axis=0)

        mha_out = self.cond_attention(time_embed_unsqueeze, cond_embed).squeeze(axis=1)
        mlp_out = self.cond_mlp(mha_out)

        # pooling the conditions
        pool_out = self.pool_mlp(jnp.expand_dims(mlp_out, axis=2))
        pool_out = jnp.expand_dims((pool_out + time_embed), axis=1)

        cond_signal = jnp.expand_dims(self.linear(mlp_out), axis=1)
        cond_signal = jnp.broadcast_to(
            (cond_signal + pool_out), shape=(-1, x.shape[1], -1)
        )

        x = x + cond_signal
        x = self.patch_mixer(x)

        if mask is not None:
            x = remove_masked_patches(x, mask)

        mlp_out_us = jnp.expand_dims(mlp_out, axis=1)  # unqueezed mlp output
        cond = jnp.broadcast_to((mlp_out_us + pool_out), shape=(-1, x.shape[1], -1))

        x = x + cond

        x = self.ditbackbone(x, time_embed, cond_embed)

        x = self.final_linear(x)

        # add back masked patches
        if mask is not None:
            x = add_masked_patches(x, mask)

        x = self.ditbackbone.unpatchify(x)

        return x

    # ahhhhhhhh, yess, full model
    # wed_nov_13, 4:49

    def sample(self, z_latent, cond, sample_steps=50, cfg=2.0):
        b_size = z_latent.shape[0]
        dt = 1.0 / sample_steps

        dt = jnp.array([dt] * b_size)
        dt = jnp.reshape(dt, shape=(b_size, *([1] * len(z_latent.shape[1:]))))

        images = [z_latent]

        for i in range(sample_steps, 0, -1):
            t = i / sample_steps
            t = (
                jnp.array([t] * b_size)
                # .to_device(z_latent.device)
                .astype(z_latent.dtype)
            )

            vc = self(z_latent, t, cond, None)
            null_cond = jnp.zeros_like(cond)
            vu = self.__call__(z_latent, t, null_cond)
            vc = vu + cfg * (vc - vu)

            z = z_latent - dt * vc
            images.append(z)

        return images[-1] / config.vaescale_factor

randin = jrand.normal(randkey, (1, 4, 32, 32))
microdit = MicroDiT(
    inchannels=3,
    patch_size=(4, 4),
    embed_dim=1024,
    num_layers=4,
    attn_heads=6,
    mlp_dim=4 * 1024,
    cond_embed_dim=768,
    )

sincos <class 'jaxlib.xla_extension.ArrayImpl'> (1, 64, 1024)


/usr/local/lib/python3.10/dist-packages/jax/_src/numpy/lax_numpy.py:5037: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in arange is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  output = _arange(start, stop=stop, step=step, dtype=dtype)


  0%|          | 0/4 [00:00<?, ?it/s]

dit block online
dit block online
dit block online


ValueError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 16777732 bytes.

In [109]:
import gc

jax.clear_caches()

gc.collect()

807

In [110]:
# rectifed flow forward pass, loss, and smapling
class RectFlow(nnx.Module):
    def __init__(self, model: nnx.Module, sigln: bool = True):
        self.model = model
        self.sigln = sigln

    def __call__(self, x_input: Array, cond: Array, mask) -> Array:

        b_size = x_input.shape[0]  # batch_sie
        rand_t = None

        if self.sigln:
            rand = jrand.normal(randkey, (b_size,))#.to_device(x_input.device)
            rand_t = nnx.sigmoid(rand)
        else:
            rand_t = jrand.normal(randkey, (b_size,))#.to_device(x_input.device)

        inshape = [1] * len(x_input.shape[1:])
        texp = rand_t.reshape([b_size, *(inshape)])

        z_noise = jrand.normal(
            randkey, x_input.shape
        )  # input noise with same dim as image
        z_noise_t = (1 - texp) * x_input + texp * z_noise

        v_thetha = self.model(z_noise_t, rand_t, cond, mask)

        mean_dim = list(
            range(1, len(x_input.shape))
        )  # across all dimensions except the batch dim
        mean_square = (z_noise - x_input - v_thetha) ** 2  # squared difference
        batchwise_mse_loss = jnp.mean(mean_square, axis=mean_dim)  # mean loss

        return jnp.mean(batchwise_mse_loss)

    def sample(
        self,
        input_noise: jax.Array,
        cond,
        zero_cond=None,
        sample_steps: int = 50,
        cfg=2.0,
    ) -> List[jax.Array]:

        batch_size = input_noise.shape[0]

        # array reciprocal of sampling steps
        d_steps = 1.0 / sample_steps

        d_steps = jnp.array([d_steps] * batch_size)#.to_device(input_noise.device)
        steps_dim = [1] * len(input_noise.shape[1:])
        d_steps = d_steps.reshape([batch_size], *steps_dim)

        images = [input_noise]  # noise sequence

        for t_step in range(sample_steps):

            genstep = t_step / sample_steps  # current step

            genstep_batched = jnp.array([genstep] * batch_size)#.to_device(input_noise.device)

            cond_output = self.model(
                input_noise, genstep_batched, cond
            )  # get model output for step

            if zero_cond is not None:
                # output for zero conditioning
                uncond_output = self.model(input_noise, genstep_batched, zero_cond)
                cond_output = uncond_output + cfg * (cond_output - uncond_output)

            out_noise = input_noise - d_steps * cond_output

            images.append(out_noise)

        return images


# model = MicroDiT(
#     inchannels=3,
#     patch_size=(4, 4),
#     embed_dim=1024,
#     num_layers=12,
#     attn_heads=6,
#     mlp_dim=4 * 1024,
#     caption_embed_dim=768,
# )

rf_engine = RectFlow(microdit)
graph, state = nnx.split(rf_engine)
n_params = sum([p.size for p in jax.tree.leaves(state)])
print(f"number of parameters: {n_params/1e6:.3f}M")

# optimizer = nnx.Optimizer(rf_engine, optax.adamw(learning_rate=config.lr))

# def preprocess_image(image_array):

def wandb_logger(key: str, project_name, run_name):  # wandb logger
    # initilaize wandb
    wandb.login(key=key)
    wandb.init(project=project_name, name=run_name)

number of parameters: 482.471M


In [111]:
def loss_func(model, batch):
    img_latents, label = batch
    bs, height, width, channels = img_latents.shape
    print(img_latents.shape)

    img_latents = img_latents * config.vaescale_factor
    mask = random_mask(
        bs, height, width, patch_size=config.patch_size, mask_ratio=config.mask_ratio
    )#.to_device(img_latents.device)
    print(f'mask shape {mask.shape}')
    loss = model(img_latents, label, mask)
    # loss = optax.squared_error(img_latents, logits).mean()

    return loss


@nnx.jit
def train_step(model, optimizer, batch):
    gradfn = nnx.value_and_grad(loss_func, has_aux=True)
    (loss, logits), grads = gradfn(model, batch)
    optimizer.update(grads)
    return loss


def sample_images(model, vae, noise, embeddings):
    # Use the stored embeddings
    sampled_latents = model.sample(noise, embeddings)

    # Decode latents to images
    sampled_images = vae.decode(sampled_latents).sample
    # images = sample_images
    return sampled_images


def trainer(model=rf_engine, optimizer=optimizer, train_loader=train_loader):
    epochs = 2
    train_loss = 0.0
    model.train()
    # wandb_logger(
    #     key=None,
    #     model=model,
    #     project_name="transformer_playjax",
    #     run_name="tinygpt-1e-4-bs32-tpu",
    # )

    for epoch in tqdm(range(epochs)):
        for step, batch in tqdm(enumerate(train_loader), total=len(train_loader)):

            train_loss = train_step(model, optimizer, batch)
            print(f"step {step}, loss-> {train_loss.item():.4f}")

            # if step % 100 == 0:
            #     sample_latents = model.sample()

            # wandb.log({"loss": train_loss.item()})

        print(f"epoch {epoch+1}, train loss => {train_loss}")
        save_model(model.model, 'microdit_cifar1.safetensors')


trainer()
# wandb.finish()
print("microdit test training in JAX")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

(4, 32, 32, 3)
mask shape (4, 64)
patchembed shape => (4, 65536)


/usr/local/lib/python3.10/dist-packages/jax/_src/numpy/lax_numpy.py:5037: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in arange is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  output = _arange(start, stop=stop, step=step, dtype=dtype)


ValueError: Incompatible shapes for broadcasting: (1, 64, 1024) and requested shape (4, -1, -1)